In [1]:
import os
import sys
sys.path.append('../')

import numpy as np
import matplotlib
from matplotlib.dates import DateFormatter
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from pprint import pprint
from scipy.optimize import curve_fit

import src.io as sio
import src.preprocessing as spp
import src.fitting as sft
import ipympl
import pytz

In [2]:
BAKEOUT_FOLDER = sio.get_folderpath("20201030_Bakeout")

In [3]:
df = sio.read_tm224_data("temperature-monitoring04.xls", BAKEOUT_FOLDER)

fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a")
ax.xaxis.set_major_formatter(myFmt)

ax.plot(df["MPL_datetimes"], df["Baseplate"])

x, y = sft.time_extrapolation(df, ylabel="Baseplate", end_date="10-Nov-20 10:00", start_index=1500, fit="linear")
ax.plot(x, y, "--")
x, y = sft.time_extrapolation(df, ylabel="Baseplate", end_date="10-Nov-20 10:00", start_index=2500, fit="linear")
ax.plot(x, y, "--")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
df = sio.read_tpg_data("pressure-monitoring02", BAKEOUT_FOLDER)

fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a")
ax.xaxis.set_major_formatter(myFmt)
ax.set_yscale("log")

ax.set_ylabel(r"Pressure (mbar)")

ax.plot(df["MPL_datetimes"], df["Prep"], "-", label="Main")
ax.legend()

x, y = sft.time_extrapolation(df, ylabel="Prep", end_date="10-Nov-20 10:00", start_index=0, fit="linear")
ax.plot(x, y, "--")

#ax.set_xlim((0, 0.045))
#sio.savefig("pressure-monitoring01_02", SECOND_PUMPDOWN)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …